In [1]:
# https://platform.olimpiada-ai.ro/problems/65

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [2]:
train = pd.read_csv("/kaggle/input/detect-santa/train.csv")
test = pd.read_csv("/kaggle/input/detect-santa/test.csv")

train.shape, test.shape

((614, 2), (616, 1))

In [3]:
train.head()

,image_path,label
0,train/img_000001.jpg,1
1,train/img_000002.jpg,1
2,train/img_000003.jpg,1
3,train/img_000004.jpg,0
4,train/img_000005.jpg,1


In [4]:
train['image_path'] = '/kaggle/input/detect-santa/' + train['image_path']
test['image_path'] = '/kaggle/input/detect-santa/' + test['image_path']

In [5]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("zero-shot-image-classification", model="openai/clip-vit-base-patch32")

2025-12-26 07:07:45.972349: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766732866.249749      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766732866.328529      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766732866.958315      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766732866.958371      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766732866.958374      55 computation_placer.cc:177] computation placer alr

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cpu


In [6]:
class2idx = {
    'Santa': 1,
    'Not Santa': 0
}

candidate_labels = ['Santa', 'Not Santa']

output = pipe(
    train['image_path'][3],
    candidate_labels=candidate_labels,
)

chosen = output[0]['label']
chosen, class2idx[chosen]

('Not Santa', 0)

In [7]:
from sklearn.metrics import f1_score

y_pred, y_target = [], []


for i, row in (pbar := tqdm(train.iterrows(), total=len(train))):
    pred = class2idx[pipe(row['image_path'], candidate_labels=candidate_labels)[0]['label']]
    y_target.append(row['label'])
    y_pred.append(pred)
    pbar.set_postfix({'f1': f'{f1_score(y_target, y_pred):.5f}'})

print(f'F1 Score: {f1_score(y_target, y_pred):.5f}')

  0%|          | 0/614 [00:00<?, ?it/s]

F1 Score: 0.94671


In [8]:
y_pred = []

for i, row in (pbar := tqdm(test.iterrows(), total=len(test))):
    pred = class2idx[pipe(row['image_path'], candidate_labels=candidate_labels)[0]['label']]
    y_pred.append(pred)

subm = pd.read_csv("/kaggle/input/detect-santa/test.csv")
subm['label'] = y_pred

subm.to_csv("submission.csv", index=False)
subm.head()

  0%|          | 0/616 [00:00<?, ?it/s]

,image_path,label
0,test/img_000615.jpg,0
1,test/img_000616.jpg,0
2,test/img_000617.jpg,1
3,test/img_000618.jpg,1
4,test/img_000619.jpg,1
